In [62]:
# !pip install langchain langchain_core langchain_community langgraph

In [63]:
import operator
from typing import Annotated, TypedDict
from langchain_core.pydantic_v1 import BaseModel
from langgraph.constants import Send
from langgraph.graph import END, StateGraph, START

In [64]:
import socket
import re

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

SVC_ACC = !(gcloud config get-value core/account)
SVC_ACC = SVC_ACC[0]

PROJECT_NUMBER=str(re.search(r'\d+', SVC_ACC).group())

LOCATION="us-central1"

UNIQUE_PREFIX = socket.gethostname()
UNIQUE_PREFIX = re.sub('[^A-Za-z0-9]+', '', UNIQUE_PREFIX)

BUCKET_NAME = f"{PROJECT_ID}-{UNIQUE_PREFIX}-{LOCATION}"

BUCKET_URI = f"gs://{BUCKET_NAME}"  # @param {type:"string"}

! gcloud config set project $PROJECT_ID
! gcloud storage buckets create {BUCKET_URI} --project={PROJECT_ID} --location={LOCATION}
! mkdir output

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

Updated property [core/project].
Creating gs://my-project-0004-346516-pytorch112kagglewbi-us-central1/...
ERROR: (gcloud.storage.buckets.create) HTTPError 409: Your previous request to create the named bucket succeeded and you already own it.
mkdir: cannot create directory ‘output’: File exists


In [95]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import tool
# from langchain_vertexai import ChatGemini
from crewai_tools.tools import FileReadTool
import os, requests, re, mdpdf, subprocess
from vertexai.preview.vision_models import ImageGenerationModel
from langchain_google_vertexai import ChatVertexAI
import uuid, os

# Initialize Gemini LLM
llm = ChatVertexAI(
    model_name='gemini-1.5-pro-001', #"gemini-1.0-pro-002", # Replace with your desired Gemini model
    # model_name='gemini-1.5-flash', #"gemini-1.0-pro-002", # Replace with your desired Gemini model
    project_id=os.getenv(PROJECT_ID), # Your Vertex AI project ID
    location="us-central1", # Your Vertex AI location
    stream=True,
)


In [96]:
# from google.colab import userdata
import os

In [97]:
model = llm #

In [98]:
# Define the prompts
step1_prompt = """Step 1: I have a problem related to {input}.
Could you brainstorm three distinct solutions?
Please consider a variety of factors such as {perfect_factors}"""

step2_prompt = """Step 2: For each of the three proposed solutions, evaluate their potential.
Consider their pros and cons, initial effort needed, implementation difficulty, potential challenges,
and the expected outcomes. Assign a probability of success and a confidence level
to each option based on these factors.

Solutions:
{solutions}"""

step3_prompt = """Step 3: For each solution, deepen the thought process.
Generate potential scenarios, strategies for implementation, any necessary partnerships or resources,
and how potential obstacles might be overcome. Also, consider any potential unexpected outcomes
and how they might be handled.

Evaluation:
{review}"""

step4_prompt = """Step 4: Based on the evaluations and scenarios,
rank the solutions in order of promise. Provide a justification for each ranking
and offer any final thoughts or considerations for each solution.

Detailed analysis:
{deepen_thought_process}"""

In [99]:
# Define the output structures
class Solutions(BaseModel):
    solutions: list[str]

class Review(BaseModel):
    review: str

class DeepThought(BaseModel):
    deep_thought: str

class RankedSolutions(BaseModel):
    ranked_solutions: str

# Define the overall state
class OverallState(TypedDict):
    input: str
    perfect_factors: str
    solutions: Annotated[list[str], operator.add]
    reviews: Annotated[list[str], operator.add]
    deep_thoughts: Annotated[list[str], operator.add]
    ranked_solutions: str

# Define the state for individual solution processing
class SolutionState(TypedDict):
    solution: str

In [100]:
# Graph components
def generate_solutions(state: OverallState):
    prompt = step1_prompt.format(input=state["input"], perfect_factors=state["perfect_factors"])
    response = model.with_structured_output(Solutions).invoke(prompt)
    return {"solutions": response.solutions}

def evaluate_solution(state: SolutionState):
    prompt = step2_prompt.format(solutions=state["solution"])
    response = model.with_structured_output(Review).invoke(prompt)
    return {"reviews": [response.review]}

def deepen_thought(state: SolutionState):
    prompt = step3_prompt.format(review=state["solution"])
    response = model.with_structured_output(DeepThought).invoke(prompt)
    return {"deep_thoughts": [response.deep_thought]}

def rank_solutions(state: OverallState):
    deep_thoughts = "\n\n".join(state["deep_thoughts"])
    prompt = step4_prompt.format(deepen_thought_process=deep_thoughts)
    response = model.with_structured_output(RankedSolutions).invoke(prompt)
    return {"ranked_solutions": response.ranked_solutions}

# Define the mapping logic
def continue_to_evaluation(state: OverallState):
    return [Send("evaluate_solution", {"solution": s}) for s in state["solutions"]]

def continue_to_deep_thought(state: OverallState):
    return [Send("deepen_thought", {"solution": r}) for r in state["reviews"]]

In [101]:
# Construct the graph
graph = StateGraph(OverallState)

graph.add_node("generate_solutions", generate_solutions)
graph.add_node("evaluate_solution", evaluate_solution)
graph.add_node("deepen_thought", deepen_thought)
graph.add_node("rank_solutions", rank_solutions)

graph.add_edge(START, "generate_solutions")
graph.add_conditional_edges("generate_solutions", continue_to_evaluation, ["evaluate_solution"])
graph.add_conditional_edges("evaluate_solution", continue_to_deep_thought, ["deepen_thought"])
graph.add_edge("deepen_thought", "rank_solutions")
graph.add_edge("rank_solutions", END)

In [102]:
app = graph.compile()

In [106]:
# Call the graph
try:
    for s in app.stream({
        "input": "improving health care in a developing city",
        "perfect_factors": "cost, efficiency, environmental impact, and user experience"
    }):
        print(s)
finally:  # Replace 'ExceptionType' with the specific exception you want to catch
    # Code to handle the exception
    print(f"An error occurred")

{'generate_solutions': {'solutions': ['1. **Implement a community health worker program:** Train and employ local residents to provide basic healthcare services and education in their communities. This is a cost-effective way to increase healthcare access, especially in remote areas, and can empower individuals to take charge of their health.\\n2. **Leverage mobile technology:** Develop mobile health (mHealth) applications for disease surveillance, appointment scheduling, and telehealth consultations. This can improve efficiency, reduce travel costs, and overcome geographical barriers.\\n3. **Invest in sustainable infrastructure:** Build energy-efficient healthcare facilities powered by renewable energy sources. This reduces the environmental impact of healthcare delivery and ensures long-term sustainability.']}}


NameError: name 'e' is not defined

In [ ]:
from pprint import pprint

In [ ]:
pprint(s)

In [ ]:
from IPython.display import Image

Image(app.get_graph().draw_mermaid_png())